In [53]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px


In [55]:
import requests

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
r = requests.get(url)

with open("spacex_launch_dash.csv", "wb") as f:
    f.write(r.content)



In [57]:
import pandas as pd

spacex_df = pd.read_csv("spacex_launch_dash.csv")
spacex_df.head()


,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [59]:
spacex_df = spacex_df[spacex_df['Booster Version'] != 'Falcon 1']

# Check dataset
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [61]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px


In [63]:
# Step 5: Prepare min/max payload and unique launch sites
min_payload = spacex_df['Payload Mass (kg)'].min()
max_payload = spacex_df['Payload Mass (kg)'].max()
launch_sites = spacex_df['Launch Site'].unique()

# Step 6: Create Dash app
app =Dash(__name__)

In [65]:
# App layout
app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard", style={'textAlign':'center'}),

    # Dropdown for launch site selection
    dcc.Dropdown(
        id='site-dropdown',
        options=[{'label':'All Sites','value':'ALL'}] +
                [{'label':site,'value':site} for site in spacex_df['Launch Site'].unique()],
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),

    # Pie chart for success counts
    dcc.Graph(id='success-pie-chart'),

    # Payload slider
    html.H5("Payload Range (Kg)"),
    dcc.RangeSlider(
        id='payload-slider',
        min=0, max=10000, step=1000,
        value=[spacex_df['Payload Mass (kg)'].min(), spacex_df['Payload Mass (kg)'].max()],
        marks={0:'0', 2500:'2500', 5000:'5000', 7500:'7500', 10000:'10000'}
    ),

    # Scatter chart for payload vs. outcome
    dcc.Graph(id='success-payload-scatter-chart')
])

# Callback for pie chart
@app.callback(
    Output('success-pie-chart','figure'),
    Input('site-dropdown','value')
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        fig = px.pie(spacex_df, names='Launch Site', values='class', title='Total Success Launches by Site')
    else:
        df_filtered = spacex_df[spacex_df['Launch Site'] == selected_site]
        counts = df_filtered['class'].value_counts().reset_index()
        counts.columns = ['Outcome','Count']
        counts['Outcome'] = counts['Outcome'].map({0:'Failure',1:'Success'})
        fig = px.pie(counts, names='Outcome', values='Count', title=f'Success vs Failure at {selected_site}')
    return fig

# Callback for scatter chart
@app.callback(
    Output('success-payload-scatter-chart','figure'),
    [Input('site-dropdown','value'), Input('payload-slider','value')]
)
def update_scatter(selected_site, payload_range):
    low, high = payload_range
    df_filtered = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) & 
                            (spacex_df['Payload Mass (kg)'] <= high)]
    
    if selected_site != 'ALL':
        df_filtered = df_filtered[df_filtered['Launch Site']==selected_site]
    
    # Safety check: if filtering removes all rows
    if df_filtered.empty:
        fig = px.scatter(title="No data available for selected filters")
    else:
        fig = px.scatter(df_filtered, x='Payload Mass (kg)', y='class', 
                         color='Booster Version Category',
                         title='Correlation between Payload and Success for selected Sites')
    return fig

# Run the app in Jupyter Notebook
app.run(mode='inline')